In [0]:
#importing modules
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
#creating spark session object
spark = SparkSession.builder.appName("Amazon_Customer_Review").getOrCreate()

In [0]:
#unmouting the existing file path for the storage account(if any)
dbutils.fs.unmount("/mnt/storagedemorudrakshmountpoint")

#mouting the new file path for the storage account (if not created or unmounted)
dbutils.fs.mount(
  source = "wasbs://amazoncustomerreviewanalyticscontainer@storagedemorudraksh.blob.core.windows.net",
  mount_point = "/mnt/storagedemorudrakshmountpoint",
  extra_configs = {"fs.azure.account.key.storagedemorudraksh.blob.core.windows.net": "SYqiIiD4XYckcONfbwR51ru0aw6KkkPanl+KxslUa1N29bkCSnY7MMoH1MSGO8eFeXLdOShzbEKx+AStt95jRg=="})

/mnt/storagedemorudrakshmountpoint has been unmounted.
Out[64]: True

In [0]:
# #creating dataframe from the data source 
df = spark.read.csv("dbfs:/mnt/storagedemorudrakshmountpoint/amazon_customer_reviews_usa.tsv", sep=r'\t', header=True, inferSchema = True)
df.display()

marketplace customer_id review_id product_id product_parent product_title product_category star_rating helpful_votes total_votes vine verified_purchase review_headline review_body review_date US 2975964 R1NBG94582SJE2 B00I01JQJM 860486164 GoPro Rechargeable Battery 2.0 (HERO3/HERO3+ only) Camera 5 0 0 N Y Five Stars ok 2015-08-31 US 23526356 R273DCA6Y0H9V7 B00TCO0ZAA 292641483 Professional 58mm Center Pinch Lens Cap for CANON 18-55mm , 55-250mm , 75-300mm , 50mm 1.4 , 85mm 1.8 , T5I , 70D , 60D , 7D , 7DII Camera 5 0 0 N Y Love it!!! Perfect, even sturdier than the original! 2015-08-31 US 52764145 RQVOXO7WUOFK6 B00B7733E0 75825744 Spy Tec Z12 Motion Activated Intelligent Security Camcorder Surveillance Camera Camera 2 1 1 N Y Another Motion Detect Fail If the words, "Cheap Chinese Junk" come to your mind when you see this, then congratulate yourself. You're pretty close. One of the most important features of a 'security camera" is the ability to detect motion and record, especially when running on battery and limited storage space. I tested the motion detect on this camera in a few different environments so far (i.e. low light, indoors, outdoors, etc..) and all I got was a SD card full of video triggers. All the testing was done with the low motion sensitivity setting. I can't even imagine what the high one would be. As others said, the low light operation is poor. I did one of my tests outdoors as the sun was setting. It was good until a point - then the camera started having seizures..it got fuzzy, went to black and white...back to color...then ultimately stabilized with lines across the screen. Another oddity I noticed was that the firmware reported in the windows config tool seemed later than the latest version posed on the Zetta site. I'm not exactly sure why that is. It gets two stars because there was one time, in a very light controlled indoor environment, where it didn't constantly trigger, so it MAY have some usefulness for your particular situation. In adequate light, the video isn't too bad (you don't need HD for most security applications). The are a variety of settings that are configurable, which is quite nice. One setting that could, and should, be configurable is the amount of time it will record after the last trigger is detected. The manual states that this is 5 minutes. So, if someone enters the picture for 30 seconds then leaves, you conceivably would have 5 minutes of recording nothing afterwards. This isn't power or memory efficient. My original thought was to return this, but I want to crack it open and look at some of the internals. I may decide to make my own. I will continue to test and report any new developments. 2015-08-31 US 47348933 R1KWKSF21PO6HO B006ZN4U34 789352955 Celestron UpClose G2 10x25 Monocular, Black (71213) Camera 5 0 0 N Y Exactly what I wanted and expected. Exactly what I wanted and expected. Perfect for hiking or carrying when you are going someplace you MAY need a quick closeup. I bought it to leave in my glovebox so I always have it with me. I've used it a few times already and couldn't be happier with it. For the price, it's definitely worth picking up if you are looking for a good monocular. 2015-08-31 US 33680700 R38H3UO1J190GI B00HUEBGMU 19067902 Vidpro XM-L Wired Lavalier microphone - 20' Audio Cable - Transducer type: Electret Condenser FOR Canon VIXIA HF R500 Camcorder Camera 5 1 1 N Y Good mic at a Good Price...Not Canon Though. I will look past the fact that they tricked me into believing this is a Canon product. It's not by Canon. It's some generic brand that i've never heard of. HOWEVER, it works surprisingly well! The sound quality is actually really good. The wire is actually super long and is perfect for indoor shooting. 2015-08-31 US 30301059 R3NPIFKLR19NQA B008MW6Y12 597683407 NIX 8 inch Hi-Res Digital Photo Frame with Motion Sensor (X08D) Camera 3 0 0 N Y The controls are pretty slow, and I can't get any videos to ... The controls are pretty slow, and I can't ge

In [0]:
#Find the total number of products present in the dataset
total_product_count = df.groupBy(f.col("product_id")).agg(f.count(f.col("product_id")).alias("total_product_count"))
total_product_count.show()

+----------+-------------------+
|product_id|total_product_count|
+----------+-------------------+
|B006ZN4U34|                 57|
|B008MW6Y12|               1397|
|B00T7JPGNG|                 41|
|B00VSIT5UE|                 81|
|B008525TFQ|                 32|
|B002Y5SN1C|                 10|
|B00EFILHB4|                203|
|B00XHSRVG8|                  2|
|B0093LPHZK|                 17|
|B00IVZ6LHW|                 49|
|B004VFGK1I|                122|
|B006T83DBK|                  3|
|B000H92BTM|                252|
|B00GAOQKWC|                 27|
|B00JNV5FN4|                 29|
|B00MCKG1FY|                 71|
|B008AGP5H4|                 24|
|B009FCGCHY|                114|
|B000VJX7DW|                240|
|B00ROSV3CA|                  2|
+----------+-------------------+
only showing top 20 rows



In [0]:
#Find the average product rating reviews for each product
avg_product_rating_review = df.groupBy(f.col("product_id")).agg(f.count("product_id").alias("count"),
                                                                f.avg("star_rating").alias("average_rating"))
avg_product_rating_review = avg_product_rating_review.select(["product_id", "count", "average_rating"])
avg_product_rating_review = avg_product_rating_review.orderBy(f.col("product_id"))
avg_product_rating_review.show()

+----------+-----+------------------+
|product_id|count|    average rating|
+----------+-----+------------------+
|0011300000|    1|               3.0|
|094339676X|    1|               5.0|
|0974096512|    1|               4.0|
|0981602940|    1|               5.0|
|0983947600|    1|               4.2|
|0984445129|    1|               5.0|
|0984445145|    1|               5.0|
|0984445161|    1|               5.0|
|0984920242|    1|               5.0|
|1123000034|    1|3.6666666666666665|
|1210300001|    1|               4.0|
|1349861049|    1|               5.0|
|1379178304|    1|3.4285714285714284|
|1439618038|    1|               2.5|
|1600593070|    1|               5.0|
|1921453575|    1|               1.0|
|1938450930|    1|               5.0|
|1948609819|    1|               4.0|
|198347598X|    1|               4.0|
|3100028120|    1|               4.0|
+----------+-----+------------------+
only showing top 20 rows



In [0]:
#Find the top N reviewed products sorted by count
topN_reviewed_products = df.groupBy(f.col("product_id")).agg(f.count("product_id").alias("count"))
topN_reviewed_products = topN_reviewed_products.orderBy(f.col("count").desc())
topN_reviewed_products.show(10)

+----------+-----+
|product_id|count|
+----------+-----+
|B006ZP8UOW| 4654|
|B00007E7JU| 4399|
|B0039BPG1A| 3619|
|B002VPE1WK| 3565|
|B0050R67U0| 3177|
|B00AAIPT76| 2358|
|B00009R6TA| 2317|
|B00007EDZG| 2269|
|B00F9FCW7K| 2244|
|B004TJ6JH6| 2234|
+----------+-----+
only showing top 10 rows



In [0]:
#Find total product rating for all those products which are present in the top N reviewed products

topN_reviewed_products_total_rating = df.select(["product_id","star_rating"])

topN_reviewed_products_total_rating = topN_reviewed_products_total_rating.groupBy(f.col("product_id")).agg(
                                                                                  f.count(f.col("product_id")).alias("count"),
                                                                                  f.avg(f.col("star_rating")).alias("rating_count"))
topN_reviewed_products_total_rating = topN_reviewed_products_total_rating.orderBy(f.col("count").desc())
topN_reviewed_products_total_rating.show(10)

+----------+-----+------------------+
|product_id|count|      rating_count|
+----------+-----+------------------+
|B006ZP8UOW| 4654| 4.007520412548345|
|B00007E7JU| 4399| 4.581041145714935|
|B0039BPG1A| 3619| 4.441282122133186|
|B002VPE1WK| 3565| 4.508835904628331|
|B0050R67U0| 3177|4.8451369216241735|
|B00AAIPT76| 2358| 4.603053435114504|
|B00009R6TA| 2317| 4.488131204143289|
|B00007EDZG| 2269| 4.387836051123843|
|B00F9FCW7K| 2244|3.9188948306595366|
|B004TJ6JH6| 2234| 4.475828111011638|
+----------+-----+------------------+
only showing top 10 rows



In [0]:
#Find all the users who has reviewed each product
user_id_review_product_id = df.select([f.col("product_id"), f.col("customer_id")])
user_id_review_product_id=user_id_review_product_id.groupBy(f.col("product_id")).agg(f.collect_list("customer_id").
                                                                                      alias("reviewed_by_customer_id"))
user_id_review_product_id= user_id_review_product_id.withColumn("reviewed_by_customer_id", f.concat_ws(", ", "reviewed_by_customer_id"))
user_id_review_product_id.show()

+----------+-----------------------+
|product_id|reviewed_by_customer_id|
+----------+-----------------------+
|0011300000|                4218977|
|0974096512|               14514479|
|0984445161|   13798903, 2646035...|
|1210300001|               36088608|
|1379178304|   20001236, 2890891...|
|1921453575|     32600367, 24166673|
|3100028120|               25054115|
|6000006179|   5479562, 9761782,...|
|6000006853|      21692340, 6380181|
|6000007388|   44270846, 1309488...|
|6000013159|               10789547|
|6000089805|   2059512, 52908855...|
|7550701709|   15244257, 9915625...|
|8470465317|   17521065, 4440678...|
|8567654653|               11659234|
|9989445257|               13111921|
|B00000DM96|   22538125, 2178182...|
|B00000G20K|   33169621, 4719644...|
|B00000I9HF|   11555966, 1367129...|
|B00000J0C7|               50411459|
+----------+-----------------------+
only showing top 20 rows



In [0]:
df.write.option("header", True).partitionBy("review_date").format("parquet").mode("overwrite").save("/FileStore/tables/parquet_date_partitioned_files")

partitioned_records_by_review_date = spark.read.parquet("/FileStore/tables/parquet_date_partitioned_files")
partitioned_records_by_review_date.display()

marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
US,29250375,R2GU5P9R59OMO2,B00O4I8DO4,942958239,"Foscam FBM3502 Digital Video Baby Monitor Plus FBCAM3502 Add-On Camera - Auto Motion Tracking with Pan/Tilt, Enhanced Night Vision, Video-Off Feature, Lullaby, Digital Zoom, Two-Way Audio, Temperature Monitor, 3.5"" LCD, 2.4 Ghz Wireless FHSS (White/Blue)",Camera,5,2,3,N,Y,Great monitor!,"This monitor is great. We previously had a monitor by summer infant, which was also good but this foscam is amazing, and the price was great. It has everything I wanted in a monitor, a camera that was able to move around, an talk back button to talk to the baby, a temperature gauge, and also plays lullabies. Plus the camera has a setting that will detect and follow movement which is great for a toddler. I love this monitor.",2015-01-03
US,780376,R160KQ74HMYSF2,B00H57HJJ6,45007752,NodArtisan Close-Up Lens for Instax Mini 7S Mini 8 and Polaroid 300 Cameras (Self-Portrait Mirror) - Blue,Camera,5,0,0,N,Y,Five Stars,useful,2015-01-03
US,15492216,R2P2XN7U6ZEDTG,B00JTURSGG,388566126,"MegaGear ''Ultra Light'' Neoprene Camera Case Bag with Carabiner for Sony A6000, A6300 with 16-50 Lens",Camera,4,0,0,N,Y,Four Stars,fits nice but not that much of protection,2015-01-03
US,30834445,RZRXYGC5J8STN,B0054LK280,947424706,Duracell DL1/3N CR1/3N 2L76 5018LC K58L 3V Lithium Battery,Camera,5,0,1,N,Y,... batteries were exactly what i needed and they work great.,These batteries were exactly what i needed and they work great.,2015-01-03
US,32346034,R2BOHG3EOHLHJP,B005IQSSJG,171923378,CowboyStudio 6 x 9 Feet Photography Studio Muslin Backdrop (W102),Camera,5,0,1,N,Y,LOVE this sheet,LOVE this sheet! It needed a little bit in our steam dryer when I first got it but other than that it works great!,2015-01-03
US,42122851,R1T7DSAOS1LLM5,B00DMS0LCO,896400873,Canon 70D Bundle,Camera,5,1,1,N,N,Love this camera.,"Love this camera. Decided to upgrade from the Rebel T3. I paid the same price for the camera at BJ's with the 18-135mm STM kit lens and got an extra Canon LP-E6 battery, a Canon 300DG gadget bag, CL-10 32GB SDHC card, EOS 70D tutorial DVD, and mini Belkin HDMI cable all for the same price it is here on Amazon. Love Amazon, but couldn't pass up that deal.",2015-01-03
US,44408733,R1KVOXJU6PJIV8,B0039BPG1A,716602542,Case Logic DCB-304 Compact System/Hybrid Camera Case (Black),Camera,5,0,0,N,Y,Great case for my point and shoot Nikon Coolpix L320,Great case for my point and shoot Nikon Coolpix L320. There is room for additional batteries and memory cards. I saw the same exact case at Walmart for $6 more.,2015-01-03
US,23629818,R1WOQRTUFA6J9I,B00O446E1C,259461103,"EEEKit Accessories Starter Kit Plus for All GoPro HERO 4 3+ 3 2 Session LCD, Bike Clamp Car Mount, Selfie Pole, Head Chest Wrist Strap",Camera,4,0,0,N,Y,Everything was great except the pole...,"Awesome kit. Everything you need to start making sweet GoPro videos. I use the suction cup one for so many shots. The pole is nice but it is not the most stable product since it is ""metal on metal."" It tends to slide when the pole handle is raised above the GoPro. This is why I gave it a 4 star rating. Just keep that in mind if you are banking on a quality pole for your unit. The other items are great though. I would purchase this again, no doubt.",2015-01-03
US,901882,R1OY78JXOG88X9,B00N5TL98U,832858637,Olympus V603001BW000 LC-60.5GL Lens Cover (Black),Camera,4,0,0,N,N,Like it though it didn't fully cover the lens,Fast delivery in 12 days after purchase. Like it though it didn't fully cover the lens. Slight gap towards the end of lens.,2015-01-03
US,51390541,R9NBA8UCIGNAE,B00COLBNTK,92611763,Manfrotto MKBFRA4-BH BeFree Compact Aluminum Travel Tripod Black,Camera,1,4,11,N,N,Clamps break if they are not made of metal. Get something of quality instead. Run from Manfrotto,"l got a manfro

In [0]:
df.write.option("header", True).partitionBy("star_rating").format("parquet").mode("overwrite").save("/FileStore/tables/parquet_rating_partitioned_files")

partitioned_records_by_star_rating = spark.read.parquet("/FileStore/tables/parquet_rating_partitioned_files")
partitioned_records_by_star_rating.display()

marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,star_rating
US,23526356,R273DCA6Y0H9V7,B00TCO0ZAA,292641483,"Professional 58mm Center Pinch Lens Cap for CANON 18-55mm , 55-250mm , 75-300mm , 50mm 1.4 , 85mm 1.8 , T5I , 70D , 60D , 7D , 7DII",Camera,0,0,N,Y,Love it!!!,"Perfect, even sturdier than the original!",2015-08-31,5
US,47348933,R1KWKSF21PO6HO,B006ZN4U34,789352955,"Celestron UpClose G2 10x25 Monocular, Black (71213)",Camera,0,0,N,Y,Exactly what I wanted and expected.,"Exactly what I wanted and expected. Perfect for hiking or carrying when you are going someplace you MAY need a quick closeup. I bought it to leave in my glovebox so I always have it with me. I've used it a few times already and couldn't be happier with it. For the price, it's definitely worth picking up if you are looking for a good monocular.",2015-08-31,5
US,33680700,R38H3UO1J190GI,B00HUEBGMU,19067902,Vidpro XM-L Wired Lavalier microphone - 20' Audio Cable - Transducer type: Electret Condenser FOR Canon VIXIA HF R500 Camcorder,Camera,1,1,N,Y,Good mic at a Good Price...Not Canon Though.,"I will look past the fact that they tricked me into believing this is a Canon product. It's not by Canon. It's some generic brand that i've never heard of. HOWEVER, it works surprisingly well! The sound quality is actually really good. The wire is actually super long and is perfect for indoor shooting.",2015-08-31,5
US,502818,R2E7A4FF0PVY5Q,B00ZKDUFBQ,555190742,"GeekPro 2.0-Inch WIFI HD 1080P 12MP Sports Camera Bundle with Bag, Waterproof Housing and Accessories (Black)(17 Items)",Camera,0,1,N,Y,Five Stars,Great camera for the price.,2015-08-31,5
US,1481233,R3R8JDQ2BF4NM,B010BZ7S2Q,129544315,Sony HDR-AZ1VR Action Cam Mini Camcorder 1080p Wi-Fi POV + Live View Remote (Certified Refurbished),Camera,0,2,N,Y,Five Stars,Product is very good and satisfactory.This seller's items are always welcome.,2015-08-31,5
US,23208852,R22ZVRDPPXIDNL,B00LBIBYBE,746593019,GoPro Hero Filters,Camera,0,0,N,Y,Five Stars,Use these all the time.,2015-08-31,5
US,50399582,RT1KLS3QSJNUT,B00KDVQF8W,304104050,LB Photography Backdrop Vinyl Customized Children Birthday Party Banner Photo Background Studio Prop Wall Decor,Camera,4,4,N,Y,Super pleased with this backdrop,"Super pleased with this backdrop!!! Put it up in my home studio and it turned out perfect for an upcoming photo session. The photo does it no justice. This was a quick picture on my phone just to get an idea of how it would photograph. Once I got it smoothed out and lined up, it is gorgeous.",2015-08-31,5
US,36700181,R222VYJL5K5IRS,B00GUZEZL4,472875794,Waterproof Camera Float (2-pack) Floating Camera Strap for Your Underwater GoPro/Panasonic Lumix/Nikon COOLPIX AW110/Canon PowerShot D20/Fujifilm FinePix/Olympus Tough/Sony - Float the Case Around Your Wrist and Save Your Device from Sinking - 1 Year Warranty,Camera,0,0,N,Y,Excellent value!,"These float wristbands for holding camera are great. they have enough size and float capability to keep any small waterproof camera from sinking. Plus, they provide excellent cushioning when in a camera bag.",2015-08-31,5
US,47818374,R31LUR7M4PQOLU,B00FB1TBKS,710827451,"Ecolink Z-Wave PIR Motion Detector, Pet Immune (PIRZWAVE2-ECO)",Camera,0,0,N,Y,great product,"very good on batteries, recommended",2015-08-31,5
US,11736306,R2QT68OZTT2YKE,B00L8827BI,145946775,"Nikon D3200 Ultimate 4 Lens Experience includes: D3200 Camera, AF-S DX NIKKOR 18-55mm f/3.5-5.6 Lens, 55-200mm F/4-5.6G ED AF-S DX Zoom-Nikkor Lens, Pro .45x Wide Angle Lens w/ Macro, Pro 2X Telephoto Lens Converter, 32GB SD Card, 52mm Filter Set & more",Camera,0,0,N,Y,Five Stars,It is a very good starter set for this camera,2015-08-31,5


In [0]:
#Find the count of the reviews grouped by date for each product
review_count_of_each_product = spark.read.parquet("/FileStore/tables/parquet_date_partitioned_files")
review_count_of_each_product = review_count_of_each_product.groupBy("review_date").agg(f.count(f.col("review_id")).alias("total_number_of_reviews"))
review_count_of_each_product.show() 

+-----------+-----------------------+
|review_date|total_number_of_reviews|
+-----------+-----------------------+
| 2015-03-09|                   2009|
| 2015-05-19|                   1817|
| 2015-04-09|                   1743|
| 2014-08-01|                   1757|
| 2015-04-24|                   1963|
| 2015-03-16|                   1961|
| 2014-02-18|                   1419|
| 2015-03-11|                   1954|
| 2015-06-15|                   1874|
| 2014-11-17|                   2027|
| 2014-07-23|                   1749|
| 2015-06-19|                   1684|
| 2013-01-11|                   1312|
| 2015-08-24|                   2173|
| 2015-04-07|                   1949|
| 2015-01-20|                   2109|
| 2015-06-22|                   1924|
| 2014-01-08|                   1471|
| 2014-12-09|                   2015|
| 2014-12-27|                   1916|
+-----------+-----------------------+
only showing top 20 rows



In [0]:
#Find the count of products for each product star rating
product_count_of_each_star_rating = spark.read.parquet("/FileStore/tables/parquet_rating_partitioned_files")
product_count_of_each_star_rating = product_count_of_each_star_rating.groupBy("star_rating").agg(f.count(f.col("product_id")).alias("total_number_of_products"))
product_count_of_each_star_rating= product_count_of_each_star_rating.filter(f.col("star_rating").isNotNull())
product_count_of_each_star_rating= product_count_of_each_star_rating.orderBy(f.col("star_rating").asc())
product_count_of_each_star_rating.show()

+-----------+------------------------+
|star_rating|total_number_of_products|
+-----------+------------------------+
|          1|                  170157|
|          2|                   90949|
|          3|                  141460|
|          4|                  336700|
|          5|                 1062706|
+-----------+------------------------+



In [0]:
#Let's prepare data for model:
amazon_ratings = (df
    .select(
        'customer_id',
        'product_parent',
        'star_rating',
    )
).cache()
amazon_ratings = amazon_ratings = amazon_ratings.dropna()

In [0]:
#Making a train_test_split:
(training, test) = amazon_ratings.randomSplit([0.8, 0.2])

In [0]:
#Finally build an ALS model:

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=2, regParam=0.01, 
          userCol="customer_id", itemCol="product_parent", ratingCol="star_rating",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 4.425784046405234


In [0]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=2, regParam=0.01, 
          userCol="customer_id", itemCol="product_parent", ratingCol="star_rating",
          coldStartStrategy="drop",
          implicitPrefs=False) #changed param!
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 8.717875118397926


In [0]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=20, #10 was by default
          maxIter=2, regParam=0.01,
          userCol="customer_id", itemCol="product_parent", ratingCol="star_rating",
          coldStartStrategy="drop",
          implicitPrefs=False)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="star_rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 6.220317074963857


In [0]:
%%time
# Generate top 10 product recommendations for each user
prodRecs = model.recommendForAllUsers(10)
prodRecs.count()

CPU times: user 838 ms, sys: 438 ms, total: 1.28 s
Wall time: 31min 31s
Out[75]: 941421

In [0]:
prodRecs_df = prodRecs.toPandas()
print(prodRecs_df.shape)
prodRecs_df.head()

(941421, 2)


,customer_id,recommendations
0,10022,"[{'product_parent': 317599201, 'rating': 10.36..."
1,10293,"[{'product_parent': 732577535, 'rating': 5.236..."
2,10468,"[{'product_parent': 164272245, 'rating': 10.32..."
3,15070,"[{'product_parent': 633313151, 'rating': 10.25..."
4,15983,"[{'product_parent': 838265554, 'rating': 25.26..."
